# ⭐图像（数据）增广
- Data Augmentation

**核心问题：训练时的数据和实际使用时的数据长得不一样！**

### 图像增广就是提前模拟各种情况

```
你的训练数据：都是在办公室拍的，光线好，角度正

实际使用时：
    可能光线暗
    可能颜色偏黄
    可能只拍到物体的一半
    可能图片是翻转的

图像增广 = 把训练数据变换一下，模拟这些情况
    让模型提前见过各种"奇怪"的图片
    这样实际使用时就不会懵了
```
---

## 二、常用的增广方法（三种最重要）

### 方法1：翻转

```
左右翻转：最常用，几乎所有任务都用
    猫朝左 → 翻转 → 猫朝右
    一张图变两张图！

上下翻转：看情况用
    猫翻过来头朝下 → 很奇怪 → 一般不用
    树叶、卫星图 → 可以用（没有上下之分）
```

### 方法2：随机裁剪

```
从图片中随机切一块出来，再缩放到固定大小

为什么？
    实际拍照时，物体可能只占画面一部分
    可能只拍到猫的头
    可能只拍到猫的尾巴
    模型应该都能认出来

具体做法：
    随机选位置
    随机选大小（占原图8%~100%）
    随机选高宽比（3/4 到 4/3）
    切出来后缩放到224×224
```

### 方法3：颜色变换

```
随机改变亮度、色调、饱和度

为什么？
    不同灯光下颜色不同（白天vs晚上）
    不同相机拍出来颜色不同
    模型不应该因为颜色变了就认不出来

具体做法：
    亮度：在0.5到1.5之间随机（变暗或变亮）
    色调：随机偏红、偏蓝、偏黄
    饱和度：颜色浓一点或淡一点
```

---

## 三、什么时候用？

```
训练时：每次读图片都随机做增广
        同一张猫的图片，每次训练看到的都不一样
        → 相当于增加了数据量

测试/推理时：不做增广！用原始图片
        → 要稳定的结果，不能加随机性
```

**这也是一种正则化方法（和Dropout类似，只在训练时用）**

---

**要点与疑惑一一回答**
# 从根本问题开始：为什么需要Resize？



## 关键问题：网络能接受不同大小的图片吗？

### 答案：不能！（至少对于全连接层来说）

```python
# ResNet的最后是这样：
卷积层 → 全局池化 → Flatten → Linear(512, 10)
                              ^^^^^^^^
     这一层要求输入必须是512个数字

# 如果图片大小不同：
图片1（224×224）→ 卷积 → 池化 → Flatten → 512个数字 ✓
图片2（300×400）→ 卷积 → 池化 → Flatten → 738个数字 ✗
                                          ^^^
       大小变了！Linear层炸了！
```

**所以你必须让所有图片变成同样大小，才能进同一个网络。**

---

## 你可能会说：那我直接ToTensor不就行了吗？

### 看看直接ToTensor会发生什么

```python
# 只用ToTensor，不Resize
transform = transforms.ToTensor()

# 你的图片：
cat1.jpg → 原始大小 1200×800  → ToTensor → Tensor(3, 1200, 800)
cat2.jpg → 原始大小 300×400   → ToTensor → Tensor(3, 300, 400)
cat3.jpg → 原始大小 640×480   → ToTensor → Tensor(3, 640, 480)

# 做成DataLoader
train_loader = DataLoader(train_dataset, batch_size=64)

# 当你训练时
for X, y in train_loader:
    # DataLoader想把64张图打包成一个batch
    # 但64张图大小都不一样！
    # 打包失败！直接报错！
    
# 报错信息：
RuntimeError: stack expects each tensor to be equal size
```

**DataLoader要把多张图片打包成一个batch，它们必须一样大！**

---

## 所以必须Resize

```python
# 正确做法：先Resize再ToTensor
transform = transforms.Compose([
    transforms.Resize(224),        # 所有图片统一变成224×224
    transforms.ToTensor(),         # 再转成Tensor
])

# 现在：
cat1.jpg → 1200×800  → Resize → 224×224 → ToTensor → Tensor(3, 224, 224)
cat2.jpg → 300×400   → Resize → 224×224 → ToTensor → Tensor(3, 224, 224)
cat3.jpg → 640×480   → Resize → 224×224 → ToTensor → Tensor(3, 224, 224)

# 做成DataLoader
for X, y in train_loader:
    # 现在64张图都是224×224
    # 打包成功！X的形状是 (64, 3, 224, 224)
    # 可以丢进网络了
```

---

## 为什么需要transforms.Compose？

### 因为你要做两件事

```python
# 如果不用Compose，你要这样写：
图片 = 读取图片()
图片 = Resize(图片)        # 手动做第1步
图片 = ToTensor(图片)      # 手动做第2步

# 每读一张图都要写两行！太麻烦了！
```

```python
# 用Compose打包：
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

# 以后只要：
图片 = transform(原始图片)  # 自动执行两步
```

**Compose就是把多个操作打包，省得每次手动做。**

---

## 完整流程图

```
原始图片文件（cat.jpg，大小1200×800）
        │
        │ 不Resize直接ToTensor会怎样？
        ↓
    ToTensor
        │
        ↓
    Tensor(3, 1200, 800)  ← 每张图大小不一样
        │
        ↓
    DataLoader想打包成batch
        │
        ↓
    报错！无法打包不同大小的图片
```

```
原始图片文件（cat.jpg，大小1200×800）
        │
        │ 先Resize再ToTensor
        ↓
    Resize(224)
        │
        ↓
    PIL图片(224×224)  ← 现在所有图都一样大了
        │
        ↓
    ToTensor
        │
        ↓
    Tensor(3, 224, 224)  ← 所有Tensor大小都一样
        │
        ↓
    DataLoader打包成batch
        │
        ↓
    (64, 3, 224, 224)  ← 成功！可以训练
```

---

## 现在加上增广

### 没有增广（最简单）

```python
# 必须做的两件事
transform = transforms.Compose([
    transforms.Resize(224),        # 必须：统一大小
    transforms.ToTensor(),         # 必须：变成数字
])
```

### 有增广（训练时多做几步变换）

```python
# 训练时：在必须做的事之前加随机操作
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),     # 额外：随机翻转
    transforms.RandomResizedCrop(224),     # 代替Resize：随机裁剪到224
    transforms.ToTensor(),   # 必须：变成数字
])
```

### RandomResizedCrop = Resize的升级版

```
Resize(224)：
    每张图都缩放到224×224
    每次结果都一样

RandomResizedCrop(224)：
    随机切一块，再缩放到224×224
    每次结果都不一样（位置随机、大小随机）
    
效果：
    都能保证最后是224×224
    但RandomResizedCrop加了随机性（增广）
```

---

## 为什么测试时用Resize+CenterCrop而不是Resize？

```python
# 为什么不直接Resize(224)？
transforms.Resize(224)
# 问题：如果图片不是正方形，会被拉变形

原图：400×300（长方形）
    ┌────────┐
    │  🐱   │  ← 猫是正常比例
    └────────┘

直接Resize(224)后：224×224（正方形）
    ┌────┐
    │ 🐱 │  ← 猫被压扁了！
    └────┘
```

```python
# 更好的方法：先Resize(256)，再CenterCrop(224)
transforms.Resize(256)
# 保持比例缩放，短边变成256

原图：400×300
Resize(256)后：341×256（还是长方形，但小了）

transforms.CenterCrop(224)
# 从中间切出224×224

    ┌─────────┐
    │ ┌─────┐ │
    │ │ 🐱  │ │  ← 从中间切，猫不变形
    │ └─────┘ │
    └─────────┘
```

---

## 总结

```
为什么不能只ToTensor？
    → 不同大小的图片无法打包成batch
    → DataLoader直接报错

为什么需要Resize？
    → 让所有图片变成同样大小（比如224×224）
    → 这样才能打包成batch

为什么需要Compose？
    → 因为要做好几步操作（Resize + ToTensor）
    → Compose把它们打包成一个流程

增广是什么？
    → 在必须做的事（Resize/ToTensor）之前
    → 额外加几个随机操作（翻转、裁剪、变色）
    → 让模型多见世面

训练和测试的区别？
    train_transform：有随机操作（增广）
    test_transform：没有随机操作（要稳定结果）
    
    但最后都要ToTensor（这是必须的）
```



-------
# 增广代码实现
**最符合直觉的顺序**

---

### 想法1：测试时直接丢图片进去不行吗？

**试试看会发生什么：**

```python
# 场景：你有一个训练好的ResNet
net = models.resnet18()

# 你想测试一张图片
from PIL import Image
img = Image.open('cat.jpg')  # 这是一张普通图片

# 直接丢进去
net(img)  # ❌ 报错！
```

**报错信息：**
```
TypeError: expected Tensor, got PIL.Image.Image
```

**第一个问题：网络只认Tensor，不认图片文件！**

---

## 那加个ToTensor总行了吧？

```python
# 给图片加ToTensor
img = Image.open('cat.jpg')         # 原始大小：1200×800
img_tensor = transforms.ToTensor()(img)  # 转成Tensor(3, 1200, 800)

# 丢进网络
net(img_tensor)  # ❌ 又报错！
```

**报错信息：**
```
RuntimeError: size mismatch
Expected input[512] but got input[2400]
```

**第二个问题：你的图片是1200×800，网络期待224×224，大小不匹配！**

---

## 那再加个Resize总行了吧？

```python
# 先Resize再ToTensor
img = Image.open('cat.jpg')
img = transforms.Resize(224)(img)      # 缩放到224×224
img_tensor = transforms.ToTensor()(img)  # 转成Tensor(3, 224, 224)

# 丢进网络
net(img_tensor)  # ❌ 还是报错！
```

**报错信息：**
```
Expected 4D tensor, got 3D tensor
```

**第三个问题：网络期待的输入形状是(batch, 3, 224, 224)，你给的是(3, 224, 224)，少了batch维度！**

---

## 终于可以了

```python
# 加上batch维度
img = Image.open('cat.jpg')
img = transforms.Resize(224)(img)
img_tensor = transforms.ToTensor()(img)
img_tensor = img_tensor.unsqueeze(0)  # 变成(1, 3, 224, 224) |用法是在指定维度上增加一个维度

# 丢进网络
output = net(img_tensor)  # ✓ 终于成功了！
```

**但这样太麻烦了！每测试一张图都要写4行代码！**

---

## 所以才需要transform

### transform就是把这些步骤自动化

```python
# 定义一次处理流程
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

# 以后每次只要一行
img = Image.open('cat.jpg')
img_tensor = test_transform(img)
img_tensor = img_tensor.unsqueeze(0)
output = net(img_tensor)
```

**transform帮你省掉重复代码！**

---

## DataLoader更进一步自动化

### 如果你要测试1000张图片呢？

```python
# 不用DataLoader：你要写循环
results = []
for i in range(1000):
    img = Image.open(f'cat{i}.jpg')
    img_tensor = test_transform(img)
    img_tensor = img_tensor.unsqueeze(0)  # 加batch维度
    output = net(img_tensor)
    results.append(output)
    #append() 是 Python列表（list） 的内置方法，核心功能是：将指定的元素添加到列表的末尾，并且会直接修改原列表（不会返回新列表
```

### 用DataLoader：自动帮你批量处理

```python
# 定义数据集和DataLoader
test_dataset = datasets.ImageFolder('data/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64)

# 自动批量处理
results = []
for X, y in test_loader:
    # X已经是(64, 3, 224, 224)了，自动加了batch维度
    output = net(X)
    results.append(output)
```

**DataLoader帮你：**
- 自动读图片
- 自动执行transform
- 自动加batch维度
- 自动打包成批

---

## 现在理解了：测试为什么也要transform

```
测试也要transform，不是为了增广
而是为了：
    1. 把图片变成Tensor（网络只认Tensor）
    2. 把图片变成224×224（网络要求的大小）
    3. 自动化这个过程（不用每次手写）
```

---

## 按认知顺序的完整代码

### 版本1：最原始（理解用）

```python
# 测试一张图片，最笨的方法
img = Image.open('cat.jpg')   # 读图片
img = transforms.Resize(224)(img) # 缩放到224×224
img = transforms.ToTensor()(img)   # 转成Tensor(3, 224, 224)
img = img.unsqueeze(0)    # 加batch维度 → (1, 3, 224, 224)
output = net(img)     # 丢进网络
```

**问题：每张图都要写这么多行！**

---

### 版本2：用Compose简化

```python
# 把重复的步骤打包
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

# 现在每张图只要两行
img = Image.open('cat.jpg')
img = test_transform(img)   # 自动执行Resize和ToTensor
img = img.unsqueeze(0)
output = net(img)
```

**改进：Resize和ToTensor合并成一个步骤**

---

### 版本3：用DataLoader全自动

```python
# 一次性定义好
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

test_dataset = datasets.ImageFolder('data/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64)

# 自动批量处理，完全不用手动
for X, y in test_loader:
    # X已经是(64, 3, 224, 224)了
    output = net(X)
    # 处理结果...
```

**改进：连unsqueeze都不用手动写了！**

---

## 训练和测试的对比

### 不增广的版本

```python
# ===== 训练和测试用同一个transform =====
transform = transforms.Compose([
    transforms.Resize(224),  # 必须：统一大小
    transforms.ToTensor(),   # 必须：变成Tensor
])

train_dataset = datasets.ImageFolder('data/train', transform=transform)
test_dataset = datasets.ImageFolder('data/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# 训练
for X, y in train_loader:
    output = net(X)  # X已经是(64, 3, 224, 224)
    # ...
```

**这就是最简单能跑的版本！**

---

### 加增广版本（提高准确率）

```python
# ===== 训练时多加随机操作（增广） =====
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),     # 新增：随机翻转
    transforms.RandomResizedCrop(224),     # 替换Resize：随机裁剪
    transforms.ColorJitter(brightness=0.5),# 新增：随机变亮度
    transforms.ToTensor(),  # 必须
])

# ===== 测试时不加随机（稳定） =====
test_transform = transforms.Compose([
    transforms.Resize(256),   # 必须
    transforms.CenterCrop(224), # 必须（防变形）
    transforms.ToTensor(),  # 必须
])

train_dataset = datasets.ImageFolder('data/train', transform=train_transform)
test_dataset = datasets.ImageFolder('data/test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
```

---

## 对比表格

| | 不增广版本 | 增广版本 |
|:---|:---|:---|
| **训练transform** | Resize + ToTensor | RandomFlip + RandomCrop + ColorJitter + ToTensor |
| **测试transform** | Resize + ToTensor | Resize + CenterCrop + ToTensor |
| **是否分开** | 不用，可以用同一个 | 必须分开，因为训练有Random |
| **准确率** | 低一些 | 高2~5% |
| **代码复杂度** | 简单 | 稍复杂 |

---

## 总结

### 第1层理解：为什么测试也要处理图片？

```
因为网络只认Tensor，只认224×224
不处理直接丢进去会报错
```

### 第2层理解：为什么用Compose？

```
因为要做好几步（Resize + ToTensor + ...）
Compose把它们打包，省得每次手写
```

### 第3层理解：为什么用DataLoader？

```
因为要测试1000张图片
DataLoader自动：读图 + transform + 打包成batch
```

### 第4层理解：为什么训练和测试要分开？

```
不增广时：可以不分开，用同一个transform
增广时：必须分开
    训练：加Random操作，让模型多见世面
    测试：不加Random，保证结果稳定
```

### 第5层理解：增广不是必须的

```
不增广 → 能跑，但准确率低
增广   → 准确率高2~5%

数据少的时候增广很重要
数据多的时候增广作用小一些
```

---

## 代码模板

```python
# ========== 导入 ==========
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# ========== 定义transform ==========
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),     # 训练时加随机操作
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(256),   # 测试时不加随机
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# ========== 加载数据 ==========
train_dataset = datasets.ImageFolder('data/train', transform=train_transform)
test_dataset = datasets.ImageFolder('data/test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# ========== 使用（和之前的训练代码完全一样） ==========
for X, y in train_loader:
    # X已经是处理好的(64, 3, 224, 224) Tensor
    output = net(X)
    # ...
```

**记住：transform的事在DataLoader读数据时就自动完成了，训练代码一个字不用改！**